In [1]:
import numpy as np
import pandas as pd

In [2]:
lenses = pd.read_csv('lenses.csv')

In [3]:
from scipy.spatial.distance import squareform, pdist

In [4]:
from scipy.sparse.csgraph import minimum_spanning_tree

In [5]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt

In [6]:
import sys

In [7]:
from sklearn.metrics.pairwise import pairwise_distances

In [48]:
from sklearn.metrics import f1_score

In [46]:
def labelClusters(clusters, target):
    mySet = set(clusters)
    resultList = []
    for i in mySet:
        myClasses = []
        for j in range(len(clusters)):
            if (clusters[j] == i):
                myClasses.append(target[j]) 
        resultList.append(max(set(myClasses), key=myClasses.count))
    
    mySetArray = []
    for i in mySet:
        mySetArray.append(i)
        
    labels = []
        
    for i in range(len(clusters)):
        for j in range(len(mySetArray)):
            if (clusters[i] == mySetArray[j]):
                labels.append(resultList[j])
    return labels

In [8]:
def gini(x):
    # (Warning: This is a concise implementation, but it is O(n**2)
    # in time and memory, where n = len(x).  *Don't* pass in huge
    # samples!)

    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g

In [9]:
def counting(arr):
    mySet = set(arr)
    myList = []
    myNewSet = []
    for i in mySet:
        myNewSet.append(i)
        
    for i in range(len(myNewSet)):
        myList.append(0)
        for j in arr:
            if (myNewSet[i]==j):
                myList[i] += 1
    return myList

In [10]:
def menorCluster(clusters):
    mySet = set(clusters)
    minimo = len(clusters)
    menor = len(clusters)
    for i in mySet:
        if((clusters.count(i)) < minimo):
            minimo = clusters.count(i)
            menor = i
            
    myList = []
    
    for j in range(len(clusters)):
        if(clusters[j]==menor):
            myList.append(j)
            
    return myList

In [14]:
def genieLinkage(data, limiar):
    
    clusters = {}
    row_index = 0
    col_index = 0
    array = []
    

    for n in range(data.shape[0]):
        array.append(n)
    
        
    clusters[0] = array.copy()

    #finding minimum value from the distance matrix
    #note that this loop will always return minimum value from bottom triangle of matrix
    for k in range(1, data.shape[0]):
        min_val = sys.maxsize
        
        if(gini(counting(clusters[k-1])) <= limiar):
            for i in range(0, data.shape[0]):
                for j in range(0, data.shape[1]):
                    if(data[i][j]<=min_val):
                        min_val = data[i][j]
                        row_index = i
                        col_index = j
        else:
            iList = menorCluster(clusters[k-1])
            for i in iList:
                for j in range(0, data.shape[0]):
                    if(j not in iList):
                        if(data[i][j]<=min_val):
                            min_val = data[i][j]
                            row_index = i
                            col_index = j

        array = clusters[k-1].copy()
        minimum = min(array[row_index],array[col_index])
        maximum = max(array[row_index],array[col_index])
        
        for n in range(len(array)):
            if(array[n]==maximum):
                array[n] = minimum
        clusters[k] = array.copy()
        
        for i in range(len(array)):
            for j in range(len(array)):
                if(array[i]==array[j]):
                    data[i][j] = sys.maxsize
                    
        if (k>(data.shape[0]*0.8)):
            return clusters[k]

In [60]:
def genie(dataset, limiar):
    
#     dataX = dataset.loc[:,dataset.columns[:-1]].values
#     dataY = dataset.loc[:,[dataset.columns[-1]]].values
    dataX = dataset.data
    dataY = dataset.target
    
    dataDistances = pairwise_distances(dataX,metric='euclidean')
    #making all the diagonal elements infinity 
    np.fill_diagonal(dataDistances,sys.maxsize)
    
    clusters = genieLinkage(dataDistances, limiar)
    
    labels = labelClusters(clusters, dataY)
    
    fmScore = f1_score(dataY, labels, average='micro')
    
    return fmScore

In [16]:
from sklearn.datasets import load_iris
iris = load_iris()

In [61]:
genie(iris, 0.4)

0.97333333333333338